Market depth (order book)
==============

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=16)

<IB connected to 127.0.0.1:7497 clientId=16>

To get a list of all exchanges that support market depth data and display the first five:

In [2]:
l = ib.reqMktDepthExchanges()
l[:5]

[DepthMktDataDescription(exchange='DTB', secType='OPT', serviceDataType='Deep'),
 DepthMktDataDescription(exchange='LSEETF', secType='STK', serviceDataType='Deep'),
 DepthMktDataDescription(exchange='SGX', secType='FUT', serviceDataType='Deep'),
 DepthMktDataDescription(exchange='IDEALPRO', secType='CASH', serviceDataType='Deep', aggGroup=4),
 DepthMktDataDescription(exchange='ARCA', secType='STK', serviceDataType='Deep')]

Let's subscribe to market depth data for EURUSD:

In [3]:
contract = Forex('EURUSD')
ib.qualifyContracts(contract)
ticker = ib.reqMktDepth(contract)

To see a live order book, an event handler for ticker updates is made that displays a dynamically updated dataframe:

In [4]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(index=range(5),
        columns='bidSize bidPrice askPrice askSize'.split())

def onTickerUpdate(ticker):
    asks = ticker.domBids
    for i in range(5):
        df.iloc[i, 0] = asks[i].size if i < len(asks) else 0
        df.iloc[i, 1] = asks[i].price if i < len(asks) else 0
    bids = ticker.domAsks
    for i in range(5):
        df.iloc[i, 2] = bids[i].price if i < len(bids) else 0
        df.iloc[i, 3] = bids[i].size if i < len(bids) else 0
    clear_output(wait=True)
    display(df)

ticker.updateEvent += onTickerUpdate

IB.sleep(15)

,bidSize,bidPrice,askPrice,askSize
0,3500000,1.18833,1.18836,2000000
1,7500000,1.18832,1.18837,9000000
2,4500000,1.18831,1.18838,7500000
3,3000000,1.1883,1.1884,3000000
4,7500000,1.18829,1.18843,7000000


Stop the market depth subscription:

In [5]:
ib.cancelMktDepth(contract)

In [6]:
ib.disconnect()